<a href="https://colab.research.google.com/github/Linafe313/Linafe313/blob/main/Zindi_Chichewa_News_Classification_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import regularizers
import pydot

In [ ]:
import pandas as pd

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from collections import Counter

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Remove url patterns
def remove_url(text): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)
 # converting return value from list to string

In [ ]:
#Clean text  

def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 

    return text2.lower()

In [ ]:
#Read training data, drop null values, count number of categories. We have under represented groups

train_data= pd.read_csv("/content/News Train.csv")
train_data.dropna(axis = 0, how ='any',inplace=True)
train_data['Num_words_text'] = train_data['Text'].apply(lambda x:len(str(x).split()))
print('===========Train Data =========')
print(train_data['Label'].value_counts())
print(len(train_data))
print('==============================')


train_data['Text'] = train_data['Text'].apply(remove_url)
train_data['Text'] = train_data['Text'].apply(clean_text)

===========Train Data =========
POLITICS                279
SOCIAL                  152
RELIGION                147
LAW/ORDER               136
SOCIAL ISSUES           134
HEALTH                  127
ECONOMY                  86
FARMING                  78
SPORTS                   49
EDUCATION                43
RELATIONSHIPS            39
WILDLIFE/ENVIRONMENT     36
OPINION/ESSAY            26
LOCALCHIEFS              25
CULTURE                  23
WITCHCRAFT               16
MUSIC                    15
TRANSPORT                11
ARTS AND CRAFTS           7
FLOODING                  7
Name: Label, dtype: int64
1436


In [ ]:
#Split data for training and validation
x_train, x_valid, y_train, y_valid = train_test_split(train_data['Text'].tolist(), train_data['Label'].tolist(), test_size=0.10,stratify = train_data['Label'].tolist(), random_state=0)

In [ ]:
print('Train data len:'+str(len(x_train)))
print('Class distribution: '+str(Counter(y_train)))
print('Valid data len:'+str(len(x_valid)))
print('Class distribution: '+ str(Counter(y_valid)))

train_input=np.asarray(x_train)
test_input = np.array(x_valid)


le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))

test_labels = le.transform(y_valid)
test_labels = np.asarray( tf.keras.utils.to_categorical(test_labels))



Train data len:1292
Class distribution: Counter({'POLITICS': 251, 'SOCIAL': 137, 'RELIGION': 132, 'LAW/ORDER': 122, 'SOCIAL ISSUES': 121, 'HEALTH': 114, 'ECONOMY': 77, 'FARMING': 70, 'SPORTS': 44, 'EDUCATION': 39, 'RELATIONSHIPS': 35, 'WILDLIFE/ENVIRONMENT': 32, 'LOCALCHIEFS': 23, 'OPINION/ESSAY': 23, 'CULTURE': 21, 'WITCHCRAFT': 15, 'MUSIC': 14, 'TRANSPORT': 10, 'FLOODING': 6, 'ARTS AND CRAFTS': 6})
Valid data len:144
Class distribution: Counter({'POLITICS': 28, 'SOCIAL': 15, 'RELIGION': 15, 'LAW/ORDER': 14, 'HEALTH': 13, 'SOCIAL ISSUES': 13, 'ECONOMY': 9, 'FARMING': 8, 'SPORTS': 5, 'WILDLIFE/ENVIRONMENT': 4, 'RELATIONSHIPS': 4, 'EDUCATION': 4, 'OPINION/ESSAY': 3, 'CULTURE': 2, 'LOCALCHIEFS': 2, 'MUSIC': 1, 'FLOODING': 1, 'TRANSPORT': 1, 'ARTS AND CRAFTS': 1, 'WITCHCRAFT': 1})


In [ ]:
print(train_labels[:3])
print(test_labels[:3])
print(train_input[:3])
print(test_input[:3])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
['radio maria itsekera nyengo mariatona wolemba richard makombe director radio maria malawi bambo joseph kimu wayilesiyi ikukwanitsa kufalitsa uthenga wabwino yesu khristu ngakhale ngongole zoposa million kwacha bambo joseph kimu amayankhula mwambo nsembe ukaristia otsekera nyengo mariatona omwe unachitikira mtendere parish dayosizi dedza bambo kimu ndiokondwa kuti anthu ambiri amamvera wayilesiyi kuphatikizapo anthu omwe mpingo katolika zomwe anati chisonyezo kuti zikulimbikitsa anthu kukonda kupemphera anali mlendo wolemekezeka mwambowo bonga akazi pali anthu ambiri omwe amathandiza wayilesiyi omwe mipingo ndip

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [ ]:
train_input[:1]

array(['radio maria itsekera nyengo mariatona wolemba richard makombe director radio maria malawi bambo joseph kimu wayilesiyi ikukwanitsa kufalitsa uthenga wabwino yesu khristu ngakhale ngongole zoposa million kwacha bambo joseph kimu amayankhula mwambo nsembe ukaristia otsekera nyengo mariatona omwe unachitikira mtendere parish dayosizi dedza bambo kimu ndiokondwa kuti anthu ambiri amamvera wayilesiyi kuphatikizapo anthu omwe mpingo katolika zomwe anati chisonyezo kuti zikulimbikitsa anthu kukonda kupemphera anali mlendo wolemekezeka mwambowo bonga akazi pali anthu ambiri omwe amathandiza wayilesiyi omwe mipingo ndipo zapangitsa kuti radio maria ikhale ikufalitsa uthenga wabwino yesu khristu kuyambira pomwe inatsegulidwa zaka zapitazoanatero bambo kimu bambo kimu anapitiliza kupemph anthu onse akufuna kwabwino komanso akhristu kuti apitilize kuthandiza komanso kuyikonda wayilesiyi kuti izikwaniritsa cholinga chake chofalitsa uthenga wabwino anthu ambiri ndikupempheni kuti mupitilize 

In [ ]:
y_train[:1]

['RELIGION']

In [ ]:
Model = Pipeline([('vect', CountVectorizer()),   
               ('tfidf', TfidfTransformer()),
               ("Sampling", SMOTE()),
               ("model", RandomForestClassifier())
              ])

Model.fit(train_input, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = Model.predict(test_input)

print('accuracy %s' % accuracy_score(test_input, y_valid))
print(classification_report(y_valid, y_pred))

accuracy 0.0
                      precision    recall  f1-score   support

     ARTS AND CRAFTS       0.00      0.00      0.00         1
             CULTURE       0.00      0.00      0.00         2
             ECONOMY       0.00      0.00      0.00         9
           EDUCATION       0.67      0.50      0.57         4
             FARMING       0.78      0.88      0.82         8
            FLOODING       0.00      0.00      0.00         1
              HEALTH       0.70      0.54      0.61        13
           LAW/ORDER       0.50      0.57      0.53        14
         LOCALCHIEFS       0.00      0.00      0.00         2
               MUSIC       1.00      1.00      1.00         1
       OPINION/ESSAY       0.00      0.00      0.00         3
            POLITICS       0.56      0.86      0.68        28
       RELATIONSHIPS       0.75      0.75      0.75         4
            RELIGION       1.00      0.73      0.85        15
              SOCIAL       0.54      0.87      0.67     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Reading the test data set submitted and cleaning it

test_data= pd.read_csv("/content/News Test.csv")
test_data['Num_words_text'] = test_data['Text'].apply(lambda x:len(str(x).split()))
print('===========Test Data =========')
print(test_data['ID'].value_counts())
print(len(test_data))
print('==============================')


test_data['Text'] = test_data['Text'].apply(remove_url)
test_data['Text'] = test_data['Text'].apply(clean_text)

===========Test Data =========
ID_xeKzHcbE    1
ID_WzGTciJE    1
ID_ShsGuVmx    1
ID_xxYXdaqw    1
ID_zNisxgGw    1
              ..
ID_jGADcQne    1
ID_kllGikKe    1
ID_IOaQZPeL    1
ID_MpMKBVeg    1
ID_wCFZNHXe    1
Name: ID, Length: 620, dtype: int64
620


In [ ]:
test_data.head(1)

,ID,Text,Num_words_text
0,ID_ADHEtjTi,abambo odzikhweza akuchuluka kafukufuku apolis...,346


In [ ]:
new_input = test_data["Text"]

In [ ]:
new_input=np.asarray(new_input)

In [ ]:
new_input[:2]

array(['abambo odzikhweza akuchuluka kafukufuku apolisi mdziko muno waonetsa kuti abambo ndiwo amadzipha anthu alionse amene amatenga miyoyo yawo podzikhweza kapena kudzipha mnjira zina wenela pasintha zedi anatchereza tidakumana sukulu chiradzulu malinga mneneri kulikulu polisi nicholas gondwa abambo ambiri amalimba mtima kudzipha kusiyana amayi koma amayi olimba mtima amakumana nkhanza zambiri koma ochepa omwe amadzichotsera moyo adatero pocheza msangulutso polankhulapo mneneri apolisi mchigawo kumpoto maurice chapola adati abambo ambiri akakumana nkhaza mbanja amathamangira kudzipha mmalo mokadandaula kunthambi polisi victim support unit chapola adati abambo khumi ndiwo amakadandaula anthu alionse kondwani machano yemwe bambo awiri adati abambo amaopa kupita kupolisi akakhala nkhawa kuopa kuukiridwa komanso chifukwa ambiri sadziwa kulankhula ndipo katswiri woona zakaganizidwe psychiatrist yemwenso mkulu chipatala john mmzinda mzuzu charles masulani mwale adati abambo ambiri akakhala

In [ ]:
predictions = Model.predict(new_input)

In [ ]:
predictions[:2]

array(['SOCIAL ISSUES', 'RELIGION'], dtype='<U20')

In [ ]:
test_data = test_data.drop("Num_words_text", axis=1)
test_data = test_data.drop("Text", axis=1)
test_data.head(2)

,ID
0,ID_ADHEtjTi
1,ID_AHfJktdQ


In [ ]:
#Adding predictions to Numpy Array
test_data['Label'] = predictions.tolist()
type(predictions)
type(test_data)

pandas.core.frame.DataFrame

In [ ]:
test_data.head(2)

,ID,Label
0,ID_ADHEtjTi,SOCIAL ISSUES
1,ID_AHfJktdQ,RELIGION


In [ ]:
submission = test_data

In [ ]:
submission.to_csv("Chichewa_DNN_ResultsV77.csv", index=False)